In [96]:
import pandas as pd
from itertools import product
# from math import log2
import time
import numpy as np
import json
import multiprocessing as mp
df = pd.read_csv("CREA_procesado2.csv")

In [97]:
tamano = 17

f = [(len(p) == tamano) for p in df["palabra"]]

df_fbl = df[f]

df_fbl

,palabra,frec_abs,frec_norm
3127,responsabilidades,4596,30.12
6059,considerablemente,2207,14.46
9358,industrialización,1285,8.42
9755,descentralización,1219,7.99
11752,perfeccionamiento,958,6.27
...,...,...,...
725850,yatromecanicistas,1,0.00
726488,yoflopoliuhcayotl,1,0.00
727944,zarigüeyasratones,1,0.00
729533,zweckrationalität,1,0.00


In [98]:
def chk_l(l: str, expr: str):

    if l in "aeiou":
        dic = {
            "a": "á",
            "e": "é",
            "i": "í",
            "o": "ó",
            "u": "ú"
        }
        ocurrencias = expr.count(l) + expr.count(dic[l])

    else:
        ocurrencias = expr.count(l)
    if ocurrencias / len(expr) > 0.56 or (l*3 in expr):
        return True

    return False
'''
dic = {}
for i in range(2, 22):

    df_1 = df[[len(p) == i for p in df["palabra"]]]

    dic2 = {}
    for c in "abcdefghijklmnopqrstuvwxyz":

        ocs = 0
        for p in df_1["palabra"]:
            if chk_l(c, p):
                ocs += 1

        if ocs > 0: dic2[c] = ocs

    if len(dic2) > 0: dic[i] = dic2


    
s = json.dumps(dic, indent=4)

with open("ocurrencias.json", "w") as f:
    f.write(s)
'''

'\ndic = {}\nfor i in range(2, 22):\n\n    df_1 = df[[len(p) == i for p in df["palabra"]]]\n\n    dic2 = {}\n    for c in "abcdefghijklmnopqrstuvwxyz":\n\n        ocs = 0\n        for p in df_1["palabra"]:\n            if chk_l(c, p):\n                ocs += 1\n\n        if ocs > 0: dic2[c] = ocs\n\n    if len(dic2) > 0: dic[i] = dic2\n\n\n    \ns = json.dumps(dic, indent=4)\n\nwith open("ocurrencias.json", "w") as f:\n    f.write(s)\n'

In [99]:
# df_fbl[[chk_l("a", p) for p in df_fbl["palabra"]]]

In [100]:
def get_combs(expr: str, char):
    '''
    Obtiene las posibles coincidencias de una letra en una palabra
    '''

    underls = expr.count(".")

    posible_chars = [char, "."]

    combs = product(posible_chars, repeat=underls)


    exps = []

    for comb in combs:

        myGen = (elem for elem in comb)
        p = ""

        for i, c in enumerate(expr):
            if c == ".":
                p += next(myGen)
            else:
                p += expr[i]

        if len(expr) > 7 and chk_l(char, p): continue
        exps.append(p)
        

    return exps

print(len(get_combs("."*20, "l")))


221650


In [101]:
def get_regex_v(char: str, comb: str):

    dic = {
            "a": "á",
            "e": "é",
            "i": "í",
            "o": "ó",
            "u": "ú"
        }

    r = ""
    for c in comb:
        
        if c == ".":
            r += f"(?!{char}|{dic[char]})."

        elif c == char:
            r += f"[{char}{dic[char]}]"
        else:
            r += c
    
    return r

# get_regex_v("a", "g.aia.")

In [102]:
def get_regex_c(char: str, comb: str):
    
    r = ""
    for c in comb:
        
        if c == ".":
            r += f"(?!{char})."

        else:
            r += c
    
    return r
    
# get_regex_c("l", "g.il.")

In [103]:
def get_regex(char: str, comb: str):
    
    if char in "aeiou" and char != "":
        return get_regex_v(char, comb)

    else:
        return get_regex_c(char, comb)

get_regex("s", "g.st..a")

'g(?!s).st(?!s).(?!s).a'

In [104]:
len(df_fbl.index)

4146

In [105]:
def get_punt(args):

        char, expr, df = args

        # posible combinations of the letter in the expresion
        combs = get_combs(expr, char)

        # original rows of the dataframe
        rows = len(df.index)

        punt = 0

        # Get the puntuation for the combination "_______"
        regx = get_regex(char, len(expr)*".")
        df2 = df[df["palabra"].str.match(regx) == True]
        p = len(df2.index) / rows

        if p != 0:
            punt += (p * np.log2(1/p))


        # Reduce the 
        if char in "aeiou":
            vocals = {
                "a":"á",
                "e":"é",
                "i":"í",
                "o":"ó",
                "u":"ú"
            }

            df = df[[(char in p or vocals[char] in p) for p in df["palabra"]]]
        else:
            df = df[[char in p for p in df["palabra"]]]
        
        for comb in combs:
            
            regx = get_regex(char, comb)

            df2 = df[df["palabra"].str.match(regx) == True]

            reduced_rows = len(df2.index)

            p = reduced_rows / rows

            if p == 0:
                continue

            info_bits = np.log2(1/p)

            punt += info_bits * p

        return punt

In [106]:
f = [not "a" in expr for expr in df_fbl["palabra"]]
df_fbl2 = df_fbl[f]
df_fbl2

,palabra,frec_abs,frec_norm
14976,inconscientemente,691,4.52
16364,sorprendentemente,612,4.01
16429,electrodomésticos,608,3.98
24653,indefectiblemente,342,2.24
27288,circunscripciones,295,1.93
...,...,...,...
717890,vermessungswesens,1,0.00
718252,vestenbergsgreuth,1,0.00
718947,videorreproductor,1,0.00
723090,wertdringlichkeit,1,0.00


In [107]:
#def get_puntuations(expr: str):

dic = {}

for i in range(5, 6):

    t1 = time.time()

    tamano = i

    f = [len(p) == tamano for p in df["palabra"]]

    df_fbl = df[f]

    

    letras = [l for l in "abcdefghijklmnñopqrstuvwxyz"]

    args_ = [(c, "."* tamano, df_fbl) for c in letras]
    
    with mp.Pool(6) as p:

        punts = p.map(get_punt, args_)


    lst = list(zip(letras, punts))
    print(lst)


    dic2 = {tup[0]: tup[1] for tup in lst}

    dic[i] = dic2

    t2 = time.time()

    print(i, t2 - t1)

    #lst = sorted(list(dic2.items()), reverse=True, key=lambda l: l[1])
    #print(lst[:3])
    #print(dic2)
        

[('a', 2.666139737800694), ('b', 0.7983909088361276), ('c', 1.1110050797459612), ('d', 0.9395661173101545), ('e', 2.177383006036795), ('f', 0.524326960991265), ('g', 0.7323294567803701), ('h', 0.752445609157393), ('i', 1.8937245533547618), ('j', 0.3778345479823218), ('k', 0.5900744180468861), ('l', 1.392756040620288), ('m', 1.0233649387192973), ('n', 1.470819859733589), ('ñ', 0.10993232416953255), ('o', 1.8720615276615138), ('p', 0.7973906688646231), ('q', 0.12087178572642447), ('r', 1.652194975037789), ('s', 1.585829820703209), ('t', 1.2438154639393277), ('u', 1.2579753419029074), ('v', 0.4870652741196262), ('w', 0.21914235522026262), ('x', 0.22996182561519501), ('y', 0.6336666462869587), ('z', 0.4666827438816035)]
5 2.6155922412872314


In [108]:
'''
dic_def = {}

for i in range(5, 6):
    d = {lst[0]:lst[1] for lst in dic[i]}

    dic_def[i] = d
'''
s = json.dumps(dic, indent=4)

with open("puntuaciones16.json", "w") as f:
    f.write(s)

In [109]:
with open("puntuaciones2a13.json") as f:
    s = f.read()

with open("puntuaciones12a15.json") as f:
    s2 = f.read()

d1 = json.loads(s)
d2 = json.loads(s2)

d3 = d1 | d2

s = json.dumps(d3, indent=4)

with open("puntuaciones2a15.json", "w") as f:
    f.write(s)


FileNotFoundError: [Errno 2] No such file or directory: 'puntuaciones2a13.json'

In [ ]:
"" in "aeiou"

True

In [ ]:
e = "a...."
get_regex("a", e)

'a(?!).(?!).(?!).(?!).'

In [ ]:
df_r = df[[len(p) == 11 for p in df["palabra"]]]

expr = ".a......a.."

df_r2 = df_r[df_r.palabra.str.match(get_regex_c("a", expr))]

df_r2



In [ ]:
def get_puntuations(expr: str, letras: str, df: pd.DataFrame):

    df = df[[len(p) == len(expr) for p in df["palabra"]]]

    args_ = [(c, expr, df) for c in letras]

    with mp.Pool(6) as p:

        punts = p.map(get_punt, args_)


    lst = list(zip(letras, punts))

    dic2 = {tup[0]: tup[1] for tup in lst}

    lst = sorted(list(dic2.items()), reverse=True, key = lambda l: l[1])
    print()
    #best_letter = ""
    #best_punt = 0

    #for l, punt in dic2.items():
    #    if punt > best_punt:
    #        best_punt = punt
    #        best_letter = l

    # -> return dic2

letras = [l for l in "abcdefghijklmnñopqrstuvwxyz"]
get_puntuations(".....", letras, df)

[('a', 0.5152641447533236), ('e', 0.4517247271731491), ('i', 0.4110286591695343), ('o', 0.3994836432962012), ('r', 0.3582390687987268), ('s', 0.3402814396219338), ('n', 0.3151777132733672), ('l', 0.28383896120072893), ('u', 0.2769697758725374), ('t', 0.25521546704393555), ('c', 0.23032609838109633), ('m', 0.2103612160728021), ('d', 0.18940969913354386), ('b', 0.16247765044225682), ('p', 0.15870879253763068), ('h', 0.14628185720568584), ('g', 0.1440057617676516), ('y', 0.12115130749555561), ('k', 0.10869622338763112), ('f', 0.1003689125703882), ('v', 0.08972440671497957), ('z', 0.0809030616849592), ('j', 0.06842158666065863), ('x', 0.035429481055559695), ('w', 0.03517374409300408), ('q', 0.01766937751756101), ('ñ', 0.016027658321749863)]


In [ ]:
expr = ".an.....ar."

df_r3 = df_r2[df_r2.palabra.str.match(get_regex_c("r", expr))]

df_r3

,palabra,frec_abs,frec_norm
49930,manifestará,112,0.73
221305,manifestaré,6,0.03
309997,manifestare,3,0.01
675090,sanchiguaro,1,0.00
675649,santificará,1,0.00


In [ ]:
letras.pop(letras.index("r"))
get_puntuations(expr, letras, df_r3)

b 0.0
c 4.4217935649972375
d 0.0
f 4.643856189774724
g 2.575424759098899
h 2.575424759098899
i 0
j 0.0
k 0.0
l 0.0
m 5.28771237954945
n 0.0
ñ 0.0
o 2.575424759098899
p 0.0
q 0.0
s 0
t 4.643856189774724
u 2.575424759098899
v 0.0
w 0.0
x 0.0
y 0.0
z 0.0


In [ ]:
expr = "a..i..í.ic."

df_r4 = df_r3[df_r3.palabra.str.match(get_regex_c("a", expr))]

df_r4

,palabra,frec_abs,frec_norm
87410,ansiolítico,40,0.26
136582,antiofídico,16,0.10
334979,antiquímico,2,0.01
436322,angiolítico,1,0.00
